In [1]:
from src.estimation import estimate_text_distribution
from src.MLE import MLE
import pandas as pd
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# OJS Education

In [2]:
name = "ojs_ed"

Training data: OJS 2021 monolingual English Education research article abstracts (n = 27,010)<br><br>
Validation data: OJS Jan - Nov 2022 English Education abstracts<br>
    Monolingual: 26,094 abstracts, 196,928 sentences<br>
    Translated: 17,385 abstracts, 119,340 sentences

In [3]:
human_data_21 = pd.read_parquet(f"data/training_data/{name}/human_data_21.parquet")
print(f"human_data_21: {human_data_21.shape[0]} sentences")
human_data_21.head()

human_data_21: 212521 sentences


,identifier,date,creator,publisher,title,human_sentence,source,relation,language,rights,predicted_language,predicted_fos,description,sent_length
0,[https://educreative.id/index.php/edu/article/...,[2021-07-24],"[-, Eni Dwi Kurniawati]",[Pusat Studi Pendidikan Kreatifitas Anak Wadas...,[Peningkatan Kemampuan Guru Membuat Adminstras...,"[this, research, departs, from, the, lack, of,...",[Educreative : Jurnal Pendidikan Kreativitas A...,[https://educreative.id/index.php/edu/article/...,[eng],[Copyright (c) 2021 Educreative : Jurnal Pendi...,en,Education,[This research departs from the lack of teache...,22
1,[https://educreative.id/index.php/edu/article/...,[2021-07-24],"[-, Eni Dwi Kurniawati]",[Pusat Studi Pendidikan Kreatifitas Anak Wadas...,[Peningkatan Kemampuan Guru Membuat Adminstras...,"[it, was, discovered, when, carrying, out, coa...",[Educreative : Jurnal Pendidikan Kreativitas A...,[https://educreative.id/index.php/edu/article/...,[eng],[Copyright (c) 2021 Educreative : Jurnal Pendi...,en,Education,[This research departs from the lack of teache...,13
2,[https://educreative.id/index.php/edu/article/...,[2021-07-24],"[-, Eni Dwi Kurniawati]",[Pusat Studi Pendidikan Kreatifitas Anak Wadas...,[Peningkatan Kemampuan Guru Membuat Adminstras...,"[there, are, still, many, teachers, who, do, n...",[Educreative : Jurnal Pendidikan Kreativitas A...,[https://educreative.id/index.php/edu/article/...,[eng],[Copyright (c) 2021 Educreative : Jurnal Pendi...,en,Education,[This research departs from the lack of teache...,16
3,[https://educreative.id/index.php/edu/article/...,[2021-07-24],"[-, Eni Dwi Kurniawati]",[Pusat Studi Pendidikan Kreatifitas Anak Wadas...,[Peningkatan Kemampuan Guru Membuat Adminstras...,"[therefore, this, study, aims, to, improve, th...",[Educreative : Jurnal Pendidikan Kreativitas A...,[https://educreative.id/index.php/edu/article/...,[eng],[Copyright (c) 2021 Educreative : Jurnal Pendi...,en,Education,[This research departs from the lack of teache...,30
4,[https://educreative.id/index.php/edu/article/...,[2021-07-24],"[-, Eni Dwi Kurniawati]",[Pusat Studi Pendidikan Kreatifitas Anak Wadas...,[Peningkatan Kemampuan Guru Membuat Adminstras...,"[the, research, method, used, is, school, acti...",[Educreative : Jurnal Pendidikan Kreativitas A...,[https://educreative.id/index.php/edu/article/...,[eng],[Copyright (c) 2021 Educreative : Jurnal Pendi...,en,Education,[This research departs from the lack of teache...,9


In [4]:
ai_data_21 = pd.read_parquet(f"data/training_data/{name}/ai_data_21.parquet")
print(f"ai_data_21: {ai_data_21.shape[0]} sentences")
ai_data_21.head()

ai_data_21: 451240 sentences


,ai_sentence
0,"[the, goal, of, this, article, is, to, delve, ..."
1,"[additionally, it, aims, to, examine, the, ben..."
2,"[to, achieve, these, objectives, the, study, b..."
3,"[it, acknowledges, that, the, way, students, a..."
4,"[by, understanding, and, implementing, effecti..."


In [5]:
# call function estimate_text_distribution to get the AI content distribution & human content distribution
estimate_text_distribution(f"data/training_data/{name}/human_data_21.parquet",f"data/training_data/{name}/ai_data_21.parquet",f"distribution/{name}_21.parquet")
# load the word occurrences frequency into our framework
model=MLE(f"distribution/{name}_21.parquet")
# validate our method using mixed corpus with known ground truth alpha
# this is the guaranteed human-written evaluation set
print(f"HUMAN EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/{name}/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")
# this is the possibly google translated evaluation set
print(f"HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/ojs_ed/translated/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")

HUMAN EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.060,     0.001,     0.060
Ground Truth,Prediction,        CI,     Error
     0.025,     0.086,     0.002,     0.061
Ground Truth,Prediction,        CI,     Error
     0.050,     0.110,     0.002,     0.060
Ground Truth,Prediction,        CI,     Error
     0.075,     0.133,     0.002,     0.058
Ground Truth,Prediction,        CI,     Error
     0.100,     0.156,     0.002,     0.056
Ground Truth,Prediction,        CI,     Error
     0.125,     0.178,     0.002,     0.053
Ground Truth,Prediction,        CI,     Error
     0.150,     0.200,     0.002,     0.050
Ground Truth,Prediction,        CI,     Error
     0.175,     0.222,     0.002,     0.047
Ground Truth,Prediction,        CI,     Error
     0.200,     0.243,     0.002,     0.043


Training data: OJS 2021 monolingual English Education research article abstracts + possibly google translated abstracts (n = 48,391)<br><br>
Validation data: OJS Jan - Nov 2022 English Education abstracts<br>
    Monolingual: 26,094 abstracts, 196,928 sentences<br>
    Translated: 17,385 abstracts, 119,340 sentences

In [ ]:
human_data_21 = pd.read_parquet(f"data/training_data/{name}/human_data_21.parquet")
human_data_21_translated = pd.read_parquet(f"data/training_data/{name}/human_data_translated_21.parquet")
human_data = pd.concat([human_data_21, human_data_21_translated])
print(f"human_data: {human_data.shape[0]} sentences")
# save the data to combined parquet file
human_data.to_parquet(f"data/training_data/{name}/human_data_plus_translated_21.parquet")
# show the first few rows
human_data.head()

In [5]:
ai_data_21 = pd.read_parquet(f"data/training_data/{name}/ai_data_21.parquet")
ai_data_21_translated = pd.read_parquet(f"data/training_data/{name}/ai_data_translated_21.parquet")
ai_data = pd.concat([ai_data_21, ai_data_21_translated])
print(f"ai_data: {ai_data.shape[0]} sentences")
# save the data to new parquet files
ai_data.to_parquet(f"data/training_data/{name}/ai_data_plus_translated_21.parquet")
# show the first few rows
ai_data.head()

human_data: 354945 sentences
ai_data: 697274 sentences


In [7]:
# call function estimate_text_distribution to get the AI content distribution & human content distribution
estimate_text_distribution(f"data/training_data/{name}/human_data_plus_translated_21.parquet",f"data/training_data/{name}/ai_data_plus_translated_21.parquet",f"distribution/{name}_plus_translated_21.parquet")
# load the word occurrences frequency into our framework
model=MLE(f"distribution/{name}_plus_translated_21.parquet")
# validate our method using mixed corpus with known ground truth alpha
# this is the guaranteed human-written evaluation set
print(f"HUMAN EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/{name}/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")
# this is the possibly google translated evaluation set
print(f"HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/ojs_ed/translated/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")

HUMAN EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.080,     0.002,     0.080
Ground Truth,Prediction,        CI,     Error
     0.025,     0.110,     0.002,     0.085
Ground Truth,Prediction,        CI,     Error
     0.050,     0.138,     0.002,     0.088
Ground Truth,Prediction,        CI,     Error
     0.075,     0.164,     0.002,     0.089
Ground Truth,Prediction,        CI,     Error
     0.100,     0.188,     0.002,     0.088
Ground Truth,Prediction,        CI,     Error
     0.125,     0.213,     0.002,     0.088
Ground Truth,Prediction,        CI,     Error
     0.150,     0.238,     0.002,     0.088
Ground Truth,Prediction,        CI,     Error
     0.175,     0.260,     0.002,     0.085
Ground Truth,Prediction,        CI,     Error
     0.200,     0.285,     0.002,     0.085
Ground Truth,Prediction,        CI,     Error
     0.225,     0.308,     0.003,     0.083
Ground Truth,Prediction,        CI,     Error
     0.250,     0.331,     0.003,

Training data: OJS 2021 + 2020 monolingual English Education research article abstracts (n = 46,128)<br><br>
Validation data: OJS Jan - Nov 2022 English Education abstracts<br>
    Monolingual: 26,094 abstracts, 196,928 sentences<br>
    Translated: 17,385 abstracts, 119,340 sentences

In [8]:
human_data_21 = pd.read_parquet(f"data/training_data/{name}/human_data_21.parquet")
human_data_20 = pd.read_parquet(f"data/training_data/{name}/human_data_20.parquet")
ai_data_21 = pd.read_parquet(f"data/training_data/{name}/ai_data_21.parquet")
ai_data_20 = pd.read_parquet(f"data/training_data/{name}/ai_data_20.parquet")
# stack the data
human_data = pd.concat([human_data_21,human_data_20])
print(f"human_data: {human_data.shape[0]} sentences")
ai_data = pd.concat([ai_data_21,ai_data_20])
print(f"ai_data: {ai_data.shape[0]} sentences")
# save the data to new parquet files
human_data.to_parquet(f"data/training_data/{name}/human_data_21_20.parquet")
ai_data.to_parquet(f"data/training_data/{name}/ai_data_21_20.parquet")

human_data: 365359 sentences
ai_data: 771580 sentences


In [9]:
# call function estimate_text_distribution to get the AI content distribution & human content distribution
estimate_text_distribution(f"data/training_data/{name}/human_data_21_20.parquet",f"data/training_data/{name}/ai_data_21_20.parquet",f"distribution/{name}_21_20.parquet")
# load the word occurrences frequency into our framework
model=MLE(f"distribution/{name}_21_20.parquet")
# validate our method using mixed corpus with known ground truth alpha
# this is the guaranteed human-written evaluation set
print(f"HUMAN EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/{name}/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")
# this is the possibly google translated evaluation set
print(f"HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/ojs_ed/translated/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print( "+---------------------------------+")

HUMAN EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.059,     0.001,     0.059
Ground Truth,Prediction,        CI,     Error
     0.025,     0.085,     0.001,     0.060
Ground Truth,Prediction,        CI,     Error
     0.050,     0.109,     0.002,     0.059
Ground Truth,Prediction,        CI,     Error
     0.075,     0.133,     0.002,     0.058
Ground Truth,Prediction,        CI,     Error
     0.100,     0.155,     0.002,     0.055
Ground Truth,Prediction,        CI,     Error
     0.125,     0.178,     0.002,     0.053
Ground Truth,Prediction,        CI,     Error
     0.150,     0.200,     0.002,     0.050
Ground Truth,Prediction,        CI,     Error
     0.175,     0.221,     0.002,     0.046
Ground Truth,Prediction,        CI,     Error
     0.200,     0.244,     0.002,     0.044
Ground Truth,Prediction,        CI,     Error
     0.225,     0.265,     0.002,     0.040
Ground Truth,Prediction,        CI,     Error
     0.250,     0.286,     0.002,

Training data: OJS 2021 + 2020 monolingual English Education research article abstracts + translated abstracts (n = 83,992)<br><br>
Validation data: OJS Jan - Nov 2022 English Education abstracts<br>
    Monolingual: 26,094 abstracts, 196,928 sentences<br>
    Translated: 17,385 abstracts, 119,340 sentences

In [11]:
human_data_21 = pd.read_parquet(f"data/training_data/{name}/human_data_21.parquet")
human_data_translated_21 = pd.read_parquet(f"data/training_data/{name}/human_data_translated_21.parquet")
human_data_20 = pd.read_parquet(f"data/training_data/{name}/human_data_20.parquet")
human_data_translated_20 = pd.read_parquet(f"data/training_data/{name}/human_data_translated_20.parquet")
ai_data_21 = pd.read_parquet(f"data/training_data/{name}/ai_data_21.parquet")
ai_data_translated_21 = pd.read_parquet(f"data/training_data/{name}/ai_data_translated_21.parquet")
ai_data_20 = pd.read_parquet(f"data/training_data/{name}/ai_data_20.parquet")
# stack the data
human_data = pd.concat([human_data_21, human_data_translated_21, human_data_20, human_data_translated_20])
print(f"human_data: {human_data.shape[0]} sentences")
ai_data = pd.concat([ai_data_21, ai_data_translated_21, ai_data_20])
print(f"ai_data: {ai_data.shape[0]} sentences")
# save the data to new parquet files
human_data.to_parquet(f"data/training_data/{name}/human_data_translated_21_20.parquet")
ai_data.to_parquet(f"data/training_data/{name}/ai_data_translated_21_20.parquet")

human_data: 617675 sentences
ai_data: 1017614 sentences


In [12]:
# call function estimate_text_distribution to get the AI content distribution & human content distribution
estimate_text_distribution(f"data/training_data/{name}/human_data_translated_21_20.parquet",f"data/training_data/{name}/ai_data_translated_21_20.parquet",f"distribution/{name}_translated_21_20.parquet")
# load the word occurrences frequency into our framework
model=MLE(f"distribution/{name}_translated_21_20.parquet")
# validate our method using mixed corpus with known ground truth alpha
# this is the guaranteed human-written evaluation set
print(f"HUMAN EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/{name}/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")

HUMAN EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.090,     0.002,     0.090
Ground Truth,Prediction,        CI,     Error
     0.025,     0.121,     0.002,     0.096


KeyboardInterrupt: 

In [13]:
# this is the possibly google translated evaluation set
print(f"HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/ojs_ed/translated/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print( "+---------------------------------+")

HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.029,     0.002,     0.029
Ground Truth,Prediction,        CI,     Error
     0.025,     0.041,     0.002,     0.016
Ground Truth,Prediction,        CI,     Error
     0.050,     0.054,     0.002,     0.004
Ground Truth,Prediction,        CI,     Error
     0.075,     0.064,     0.002,     0.011
Ground Truth,Prediction,        CI,     Error
     0.100,     0.078,     0.002,     0.022
Ground Truth,Prediction,        CI,     Error
     0.125,     0.087,     0.002,     0.038
Ground Truth,Prediction,        CI,     Error
     0.150,     0.098,     0.002,     0.052
Ground Truth,Prediction,        CI,     Error
     0.175,     0.109,     0.002,     0.066


KeyboardInterrupt: 

Training data: OJS 2020 monolingual English Education research article abstracts (n = 19,118)<br><br>
Validation data: OJS Jan - Nov 2022 English Education abstracts<br>
    Monolingual: 26,094 abstracts, 196,928 sentences<br>
    Translated: 17,385 abstracts, 119,340 sentences

In [14]:
human_data_20 = pd.read_parquet(f"data/training_data/{name}/human_data_20.parquet")
print(f"human_data_20: {human_data_20.shape[0]} sentences")
ai_data_20 = pd.read_parquet(f"data/training_data/{name}/ai_data_20.parquet")
print(f"ai_data_20: {ai_data_20.shape[0]} sentences")

human_data_20: 152838 sentences
ai_data_20: 320340 sentences


In [15]:
# call function estimate_text_distribution to get the AI content distribution & human content distribution
estimate_text_distribution(f"data/training_data/{name}/human_data_20.parquet",f"data/training_data/{name}/ai_data_20.parquet",f"distribution/{name}_20.parquet")
# load the word occurrences frequency into our framework
model=MLE(f"distribution/{name}_20.parquet")
# validate our method using mixed corpus with known ground truth alpha
# this is the guaranteed human-written evaluation set
print(f"HUMAN EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/{name}/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")

HUMAN EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.062,     0.001,     0.062
Ground Truth,Prediction,        CI,     Error
     0.025,     0.089,     0.002,     0.064
Ground Truth,Prediction,        CI,     Error
     0.050,     0.113,     0.002,     0.063
Ground Truth,Prediction,        CI,     Error
     0.075,     0.137,     0.002,     0.062


KeyboardInterrupt: 

In [16]:
# this is the possibly google translated evaluation set
print(f"HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/ojs_ed/translated/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")

HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.049,     0.002,     0.049
Ground Truth,Prediction,        CI,     Error
     0.025,     0.060,     0.002,     0.035
Ground Truth,Prediction,        CI,     Error
     0.050,     0.071,     0.002,     0.021
Ground Truth,Prediction,        CI,     Error
     0.075,     0.082,     0.002,     0.007
Ground Truth,Prediction,        CI,     Error
     0.100,     0.094,     0.002,     0.006
Ground Truth,Prediction,        CI,     Error
     0.125,     0.103,     0.002,     0.022
Ground Truth,Prediction,        CI,     Error
     0.150,     0.113,     0.002,     0.037
Ground Truth,Prediction,        CI,     Error
     0.175,     0.124,     0.002,     0.051


KeyboardInterrupt: 

Training data: OJS 2021 possibly translated abstracts (n = 21,381)<br><br>
Validation data: OJS Jan - Nov 2022 English Education abstracts<br>
    Monolingual: 26,094 abstracts, 196,928 sentences<br>
    Translated: 17,385 abstracts, 119,340 sentences

In [17]:
human_data_translated_21 = pd.read_parquet(f"data/training_data/{name}/human_data_translated_21.parquet")
print(f"human_data_translated_21: {human_data_translated_21.shape[0]} sentences")
ai_data_translated_21 = pd.read_parquet(f"data/training_data/{name}/ai_data_translated_21.parquet")
print(f"ai_data_translated_21: {ai_data_translated_21.shape[0]} sentences")

human_data_translated_21: 142424 sentences
ai_data_translated_21: 246034 sentences


In [18]:
# call function estimate_text_distribution to get the AI content distribution & human content distribution
estimate_text_distribution(f"data/training_data/{name}/human_data_translated_21.parquet",f"data/training_data/{name}/ai_data_translated_21.parquet",f"distribution/{name}_translated_21.parquet")
# load the word occurrences frequency into our framework
model=MLE(f"distribution/{name}_translated_21.parquet")
# validate our method using mixed corpus with known ground truth alpha
# this is the guaranteed human-written evaluation set
print(f"HUMAN EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/{name}/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print("+---------------------------------+")

HUMAN EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.163,     0.003,     0.163
Ground Truth,Prediction,        CI,     Error
     0.025,     0.196,     0.003,     0.171


KeyboardInterrupt: 

In [19]:
# this is the possibly google translated evaluation set
print(f"HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET")
for alpha in [0,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25]:
    estimated,ci=model.inference(f"data/validation_data/ojs_ed/translated/ground_truth_alpha_{alpha}.parquet")
    error=abs(estimated-alpha)
    print(f"{'Ground Truth':>10},{'Prediction':>10},{'CI':>10},{'Error':>10}")
    print(f"{alpha:10.3f},{estimated:10.3f},{ci:10.3f},{error:10.3f}")
print( "+---------------------------------+")

HUMAN + GOOGLE TRANSLATE (?) EVALUATION SET
Ground Truth,Prediction,        CI,     Error
     0.000,     0.097,     0.003,     0.097
Ground Truth,Prediction,        CI,     Error
     0.025,     0.119,     0.003,     0.094
Ground Truth,Prediction,        CI,     Error
     0.050,     0.142,     0.003,     0.092
Ground Truth,Prediction,        CI,     Error
     0.075,     0.160,     0.004,     0.085
Ground Truth,Prediction,        CI,     Error
     0.100,     0.182,     0.004,     0.082
Ground Truth,Prediction,        CI,     Error
     0.125,     0.200,     0.004,     0.075
Ground Truth,Prediction,        CI,     Error
     0.150,     0.218,     0.004,     0.068
Ground Truth,Prediction,        CI,     Error
     0.175,     0.239,     0.004,     0.064
Ground Truth,Prediction,        CI,     Error
     0.200,     0.255,     0.004,     0.055


KeyboardInterrupt: 